# Spark 101

In this lesson we will cover the basics of working with spark dataframes, and
show how spark dataframes are different from the pandas dataframes we have
been working with.

While spark dataframes might superficially look like pandas dataframes, and
even share some of the same methods and syntax, it is important to keep in
mind they are 2 seperate types of objects, and, while spark and pandas code
might look superficially similar, it tends to be semantically very different.

We'll begin by creating the spark session:

In [1]:
import pyspark

In [3]:
# create a new session:
session = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
session

## Creating Dataframes

Spark can convert any pandas dataframe into a spark dataframe with a simple
method call. For this lesson, we will use this functionality to demonstrate
the differences between spark and pandas dataframes and explore how to work
with spark dataframes.

In [5]:
import pandas as pd
import numpy as np

np.random.seed(1349)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list('abc'), 20))
)
pandas_dataframe.head()

,n,group
0,0,c
1,1,c
2,2,b
3,3,a
4,4,c


Here we start with a simple pandas dataset, and now we will convert it to a
spark dataframe:

In [6]:
df = session.createDataFrame(pandas_dataframe)

In [7]:
df

DataFrame[n: bigint, group: string]

Notice that, while we do see the column names, we don't see the data in the
dataframe like we would with a pandas dataframe. This is because spark is
*lazy*, in that it won't show us values until it has to. For the purposes of
looking at the first few rows of our data, we can use the `.show` method.

In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [10]:
# pd.DataFrame(df)

In [13]:
# df.head()
# In the process of constructing actions here,
# note that the final .show() is going to be the thing that will 
# execute and show the action of what we are doing
# secondarily, note that everything is going to display in text-based
# ascii-like tabular data formatting
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    c|
|  1|    c|
|  2|    b|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



Like pandas dataframes, spark dataframes have a .describe method:

In [15]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



Which, also like pandas, returns another dataframe. However, since this is a
spark dataframe, we have to explicitly show it.

In [18]:
df.show(3)

+---+-----+
|  n|group|
+---+-----+
|  0|    c|
|  1|    c|
|  2|    b|
+---+-----+
only showing top 3 rows



By default spark will show the first 20 rows, but we can specify how many we
want by passing a number to `.show`.

Let's use some different data so that we have a more robust dataset:

In [19]:
from pydataset import data
mpg = data('mpg')

In [20]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [21]:
mpg = session.createDataFrame(mpg)

In [22]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



Let's look at another difference from pandas:

In [25]:
# looking at a subset of columns:
# pandas: df[[''model', 'displ', 'cyl']]
mpg.select(mpg.model, mpg.displ, mpg.cyl).show()

+------------------+-----+---+
|             model|displ|cyl|
+------------------+-----+---+
|                a4|  1.8|  4|
|                a4|  1.8|  4|
|                a4|  2.0|  4|
|                a4|  2.0|  4|
|                a4|  2.8|  6|
|                a4|  2.8|  6|
|                a4|  3.1|  6|
|        a4 quattro|  1.8|  4|
|        a4 quattro|  1.8|  4|
|        a4 quattro|  2.0|  4|
|        a4 quattro|  2.0|  4|
|        a4 quattro|  2.8|  6|
|        a4 quattro|  2.8|  6|
|        a4 quattro|  3.1|  6|
|        a4 quattro|  3.1|  6|
|        a6 quattro|  2.8|  6|
|        a6 quattro|  3.1|  6|
|        a6 quattro|  4.2|  8|
|c1500 suburban 2wd|  5.3|  8|
|c1500 suburban 2wd|  5.3|  8|
+------------------+-----+---+
only showing top 20 rows



While this expression would produce a Series of values from a pandas
dataframe, for a spark dataframe this produces a Column object, which is an
object that represents a vertical slice of a dataframe, but does not contain
the data itself.

One way to use our column objects is to use them in combination with the
`.select` method. `.select` is very powerful, and lets us specify what data we
want to see in the resulting dataframe.

In [28]:
mpg.select(mpg['displ'], mpg['class']).show(3)

+-----+-------+
|displ|  class|
+-----+-------+
|  1.8|compact|
|  1.8|compact|
|  2.0|compact|
+-----+-------+
only showing top 3 rows



In [35]:
mpg.select(mpg.displ, mpg['cyl'], 'class').show(5)

+-----+---+-------+
|displ|cyl|  class|
+-----+---+-------+
|  1.8|  4|compact|
|  1.8|  4|compact|
|  2.0|  4|compact|
|  2.0|  4|compact|
|  2.8|  6|compact|
+-----+---+-------+
only showing top 5 rows



Again, notice that we don't see any data, instead we see the new dataframe
that is produced. To see the actual data, we'll again need to use `.show`

In [ ]:
# .select will give us the set of columns we want to see
# we can reference the columns with dot notation just like pandas Series
# and if we want to see the results of our action, we can chain a .show()
# at the end of the command


Our column objects support a numer of operations, including the arithmetic
operators:

In [36]:
# take the city mileage and add 1:
# a single additive transformation of the column of data
# does not require a shuffle, just wants to add 1 to everything
mpg.cty + 1

Column<'(cty + 1)'>

Here we get back a column that represents the values from the original `hwy`
column with 1 added to them. To actually see this data, we'd need to select it
and show the dataframe.

In [37]:
mpg.select('cty', mpg.cty + 1).show(5)

+---+---------+
|cty|(cty + 1)|
+---+---------+
| 18|       19|
| 21|       22|
| 20|       21|
| 21|       22|
| 16|       17|
+---+---------+
only showing top 5 rows



Once we have a column object, we can use the `.alias` method to rename it:

In [38]:
mpg.select('cty', (mpg.cty + 1).alias('city_plus_one')).show(5)

+---+-------------+
|cty|city_plus_one|
+---+-------------+
| 18|           19|
| 21|           22|
| 20|           21|
| 21|           22|
| 16|           17|
+---+-------------+
only showing top 5 rows



we can also store column objects in variables and reference them

In [39]:
# col1: the hwy column from the mpg dataframe, 
# aliased as highway_mileage
# col2: the calculation of the highway column divided by two, 
# aliased as highway_mileage_halved
col1 = mpg.hwy.alias('highway_mileage')
col2 = (mpg.hwy / 2).alias('highway_mileage_halved')

In [40]:
col1, col2

(Column<'hwy AS highway_mileage'>,
 Column<'(hwy / 2) AS highway_mileage_halved'>)

In [41]:
mpg.select(col1, col2).show(5)

+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
|             30|                  15.0|
|             26|                  13.0|
+---------------+----------------------+
only showing top 5 rows



## Other ways to create columns

In addition to the syntax we've seen above, we can create columns with the
`col` and `expr` functions from `pyspark.sql.functions` module.

### `col`

In [42]:
from pyspark.sql import functions as F

In [44]:
# mpg.select(mpg.hwy).show(5)
hwy_col = F.col('hwy')

In [45]:
mpg.select(hwy_col).show(5)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
| 26|
+---+
only showing top 5 rows



We can mix and match the syntax we use, and the column object produced by the
`col` function is the same as the the previous column object we saw.

In [49]:
df.select(hwy_col).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `hwy` cannot be resolved. Did you mean one of the following? [`n`, `group`].;
'Project ['hwy]
+- LogicalRDD [n#0L, group#1], false


In [ ]:
# from mpg,
# select the column hwy, alias it
# select the column cty, alias it,
# select the column avg_column, alias it

In [52]:
avg_col = (2 / (
    1 / F.col('hwy').alias('highway_mileage')
) + (
    1 / F.col('cty').alias('city_mileage')
)).alias('avg_mileage')

In [54]:
mpg.select(avg_col).show(5)

+-----------------+
|      avg_mileage|
+-----------------+
|58.05555555555556|
|58.04761904761905|
|            62.05|
|60.04761904761905|
|          52.0625|
+-----------------+
only showing top 5 rows



Here we create a variable named `avg_column` that represents the average of the
highway and city mileage of each vehicle. This variable is created by using
the `col` function to produce pyspark Column objects and using the arithmetic
operators to combine them.

Next we select the original highway and city mileage columns, in addition to
our new average mileage column. We demonstrate the `col` function to select
the `hwy` column and refer to the city mileage column with the `df.cty`
syntax we saw previously. We also give all of our columns more readable
aliases before showing the resulting dataframe.

### `expr`

The `expr` function is more powerful than `col`. It does everything `col` does
and more. `expr` returns the same type of column object, but allows us to
express manipulations to the column within the string that defines the column.

In [55]:
hwy_plus = F.expr('hwy + 1 AS highway_plus_one')

In [56]:
mpg.select(hwy_plus).show(5)

+----------------+
|highway_plus_one|
+----------------+
|              30|
|              30|
|              32|
|              31|
|              27|
+----------------+
only showing top 5 rows



In [58]:
mpg.select(
    # the same as `col`
    F.expr('hwy'),
    # an arithmetic expression
    F.expr('hwy + 1'),
    # using an alias
    F.expr('hwy AS highway'),
    # a combination of the above
    F.expr('hwy + 1 AS highay_plus')
).show(5)

+---+---------+-------+-----------+
|hwy|(hwy + 1)|highway|highay_plus|
+---+---------+-------+-----------+
| 29|       30|     29|         30|
| 29|       30|     29|         30|
| 31|       32|     31|         32|
| 30|       31|     30|         31|
| 26|       27|     26|         27|
+---+---------+-------+-----------+
only showing top 5 rows



Note that all the columns created below are identical, and which syntax to use
is merely a style choice.

In [68]:
mpg.select(
    # the same as `col`
    mpg['hwy'],
    # an arithmetic expression
    F.col('hwy') + 1,
    F.expr('hwy + 1'),
    # using an alias
    mpg['hwy'].alias('highway'),
    # a combination of the above
    F.expr('hwy + 1 AS higway_plus')
).show(5)

+---+---------+---------+-------+-----------+
|hwy|(hwy + 1)|(hwy + 1)|highway|higway_plus|
+---+---------+---------+-------+-----------+
| 29|       30|       30|     29|         30|
| 29|       30|       30|     29|         30|
| 31|       32|       32|     31|         32|
| 30|       31|       31|     30|         31|
| 26|       27|       27|     26|         27|
+---+---------+---------+-------+-----------+
only showing top 5 rows



## Spark SQL

As we've seen through the column definitions, spark is very flexible and
allows us many different ways to express ourselves. Another way that is fairly
different than what we've seen above is through **spark SQL**, which lets us
write SQL queries against our spark dataframes.

In order to start using spark SQL, we'll first "register" the table with
spark:

In [69]:
mpg.createOrReplaceTempView('mpg')

Now we can write a sql query against the `mpg` table:

In [70]:
session.sql('''
SELECT cty FROM mpg LIMIT 2''').show(5)

+---+
|cty|
+---+
| 18|
| 21|
+---+



Notice that the resulting value is another dataframe. As we know, in order to
view the values in a dataframe, we need to use `.show`

In [71]:
session.sql('''
SELECT (hwy + cty) / 2 AS arithmetic_avg FROM mpg''').show(5)

+--------------+
|arithmetic_avg|
+--------------+
|          23.5|
|          25.0|
|          25.5|
|          25.5|
|          21.0|
+--------------+
only showing top 5 rows



It is worth noting that all of the methods for creating / manipulating
dataframes outlined above are the same in terms of performance as well. All of
the resulting dataframes get turned into the same spark code that gets
executed on the JVM, so it really is just a style choice as to which to use.

## Type Casting

We can view the types of the column in our dataframe in one of two ways:

In [72]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [73]:
# .printSchema() is similar to a .info() call
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



Both provide the same information.

To convert from one type to another, we can use the `.cast` method on a
column.

In [75]:
# pandas: .astype(int)
mpg.select(mpg.hwy.cast('string')).show(4)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
+---+
only showing top 4 rows



Note that if a value is not able to be converted, it will be replaced with
null:

In [76]:
mpg.select(mpg.manufacturer.cast('int')).show(5)

+------------+
|manufacturer|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+
only showing top 5 rows



## Basic Built-in Functions

We've used the `col` and `expr` functions, but there are many other functions
within the `pyspark.sql.functions` module, all of which operate on pyspark
dataframe columns. Here we'll demonstrate several:

- `concat`: to concatenate strings
- `sum`: to sum a group
- `avg`: to take the average of a group
- `min`: to find the minimum
- `max`: to find the maximum

**_Note that importing the `sum` function directly will override the built-in
`sum` function._** This means you will get an error if you try to sum a list
of numbers, because `sum` will refernce the pyspark `sum` function, which
works with pyspark dataframe columns, while the built-in `sum` function works
with lists of numbers. The same holds true for the built in `min` and `max`
functions.

In [ ]:
# from pyspark.sql.functions import lit

In [ ]:
# we can chain an alias on a concat function call
# we need to cast cylinders with a lit() function call to cast it as a string literal

In [87]:
# Note: The pyspark avg and mean functions are aliases of eachother
mpg.select(
    F.concat(
        (mpg.hwy).cast('string').alias('do_you_see_this'), 
        F.lit(' miles per gal')).alias('highway_miles'),
    F.col('manufacturer'),
    F.expr('model')
).show(5)

+----------------+------------+-----+
|   highway_miles|manufacturer|model|
+----------------+------------+-----+
|29 miles per gal|        audi|   a4|
|29 miles per gal|        audi|   a4|
|31 miles per gal|        audi|   a4|
|30 miles per gal|        audi|   a4|
|26 miles per gal|        audi|   a4|
+----------------+------------+-----+
only showing top 5 rows



In [88]:
F.expr?

Signature: F.expr(str: str) -> pyspark.sql.column.Column
Docstring:
Parses the expression string into the column that it represents

.. versionadded:: 1.5.0

.. versionchanged:: 3.4.0
    Supports Spark Connect.

Parameters
----------
str : str
    expression defined in string.

Returns
-------
:class:`~pyspark.sql.Column`
    column representing the expression.

Examples
--------
>>> df = spark.createDataFrame([["Alice"], ["Bob"]], ["name"])
>>> df.select("name", expr("length(name)")).show()
+-----+------------+
| name|length(name)|
+-----+------------+
|Alice|           5|
|  Bob|           3|
+-----+------------+
File:      ~/miniconda3/envs/homebase/lib/python3.11/site-packages/pyspark/sql/functions.py
Type:      function

!!!tip "`pyspark` imports"
    In this lesson we will explicitly import any functions from `pyspark.sql.functions` that we use, but it very common to see something like:
    
    ```
    from pyspark.sql.functions import *
    ```
    
    which will import *all* of the functions from the `pyspark.sql.functions` module.

In order to use a string literal as part of our select, we'll need to use the
`lit` function, otherwise spark will try to resolve our string as a column.

Here we select the concatenation of the number of cylinders (the value from
the `cyl` column) and the string literal " cylinders".

## More `pyspark` Functions for String Manipulation

Let's take a look at a couple more functions for string manipulation.

In [ ]:
# from pyspark.sql.functions import regexp_extract, regexp_replace

In order to demonstrate these functions we'll create a dataframe with some text data.

In [91]:
textdf = session.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)

+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



The `regexp_extract` function lets us specify a regular expression with at least one capture group, and create a new column based on the contents of a capture group.

In [94]:
# in my select call:
# address, the column as it stands
# regexp_extract, acting as a re.extract, aliasing that with a .alias method 
textdf.select(
    "address",
    F.regexp_extract('address', r'^(\d+)', 1).alias("street_no"),
    F.regexp_extract('address', r'^\d+\s([\w\s]+)', 1).alias("street"),
).show(truncate=False)

+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



In the example above, the first argument to `regexp_extract` is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.

In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression.

In [99]:
textdf.select(
    "address",
    # regexp_replace('subject', 'pattern', 'replacement)
    F.regexp_replace('address', r'^.*?,\s', '').alias("city_state_zip"),
).show(truncate=False)

+---------------------------------------------+---------------------+
|address                                      |city_state_zip       |
+---------------------------------------------+---------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |San Antonio, TX 78209|
|303 Pearl Pkwy, San Antonio, TX 78215        |San Antonio, TX 78215|
|1255 SW Loop 410, San Antonio, TX 78227      |San Antonio, TX 78227|
+---------------------------------------------+---------------------+



In our example above, we obtain just the city, state, and zip code of the address by replacing everything up to the first comma with an empty string.

## `.filter` and `.where`

Spark provides two dataframe methods, `.filter` and `.where`, which both allow
us to select a subset of the rows of our dataframe.

In [101]:
# pandas:
# mpg[mpg.cyl == 4]
mpg.filter(mpg.cyl == 4).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [104]:
mpg.filter(mpg.cyl == 4).where(F.col('class') == 'compact').show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [ ]:
# from pyspark.sql.functions import when

## When and Otherwise

Similar to an `IF` in Excel, `CASE...WHEN` in SQL, or `np.where` in python,
spark provides a `when` function.

The `when` function lets us specify a condition, and a value to produce if
that condition is true:

In [105]:
# from pyspark.sql.functions import when

In [106]:
mpg.describe().show()

23/10/25 11:01:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|             displ|             year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|               234|              234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null| 3.471794871794871|           2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.2919590310839348|4.509646313320439|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166446|null|   null|
|    min|        audi|      4runne

In [108]:
mpg.select(
    F.when(mpg.hwy > 27, 'good_mileage'
          ).otherwise(
        'bad_mileage').alias(
        'good_witch_or_bad_witch'
    )).show(5)

+-----------------------+
|good_witch_or_bad_witch|
+-----------------------+
|           good_mileage|
|           good_mileage|
|           good_mileage|
|           good_mileage|
|            bad_mileage|
+-----------------------+
only showing top 5 rows



In [110]:
session.sql('''
SELECT hwy,
CASE WHEN 
hwy > 27 THEN 'good_mileage'
ELSE 'bad_mileage'
END good_witch_bad_witch
FROM mpg
''').show()

+---+--------------------+
|hwy|good_witch_bad_witch|
+---+--------------------+
| 29|        good_mileage|
| 29|        good_mileage|
| 31|        good_mileage|
| 30|        good_mileage|
| 26|         bad_mileage|
| 26|         bad_mileage|
| 27|         bad_mileage|
| 26|         bad_mileage|
| 25|         bad_mileage|
| 28|        good_mileage|
| 27|         bad_mileage|
| 25|         bad_mileage|
| 25|         bad_mileage|
| 25|         bad_mileage|
| 25|         bad_mileage|
| 24|         bad_mileage|
| 25|         bad_mileage|
| 23|         bad_mileage|
| 20|         bad_mileage|
| 15|         bad_mileage|
+---+--------------------+
only showing top 20 rows



Notice here that if the condition we specified is false, `null` will be
produced. Instead of null, we can specify a value to use if our condition is
false with the `.otherwise` method.

To specify multiple conditions, we can chain `.when` calls. The first
condition that is met will be the value that is used, and if none of the
conditions are met the value specified in the `.otherwise` will be used (or
`null` if you don't provide a `.otherwise`).

In [112]:
mpg.select(
    F.when(mpg.hwy > 27, 'good_mileage'
          ).when(
    mpg.hwy > 18, 'ok_mileage')
    .otherwise(
        'bad_mileage').alias(
        'good_witch_or_bad_witch'
    )).show(20)

+-----------------------+
|good_witch_or_bad_witch|
+-----------------------+
|           good_mileage|
|           good_mileage|
|           good_mileage|
|           good_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|           good_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|             ok_mileage|
|            bad_mileage|
+-----------------------+
only showing top 20 rows



Notice here that a car with a `displ` of 1.8 matches both conditions we
specified, but `small` is produced because it is associated with the first
matching condition. For any value between 2 and 3, `medium` will be produced,
and anything larger than 3 will produce `large`.

## Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with
two methods: `.sort`, and `.orderBy`. `.sort` and `.orderBy` are aliases of
each other and do the exact same thing. Like other methods we've seen, `.sort`
takes in a Column object or a string that is the name of a column.

In [115]:
# note that this time when specifying ascending or descending,
# we pass it directly on the column rather than as a kwarg
#  additionally, we are just passing an mpg.sort here, not an mpg.select
mpg.sort(mpg.cyl.asc()).show(10)

+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|       model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|     hyundai|     tiburon|  2.0|1999|  4|  auto(l4)|  f| 19| 26|  r|subcompact|
|      toyota|     corolla|  1.8|1999|  4|  auto(l3)|  f| 24| 30|  r|   compact|
|       honda|       civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|      toyota|     corolla|  1.8|1999|  4|  auto(l4)|  f| 24| 33|  r|   compact|
|      subaru|forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|       suv|
|      toyota|     corolla|  1.8|1999|  4|manual(m5)|  f| 26| 35|  r|   compact|
|       honda|       civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|      toyota|     corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|   chevrolet|      malibu|  2.4|1999|  4|  auto(l4)|  f| 19| 27|  r|   midsize|
|      toyota|     corolla| 

By default, values are sorted in ascending order. To sort in descending order,
we can use the `.desc` method on any Column object, or the `desc` function
from `pyspark.sql.functions`.

In [ ]:
# from pyspark.sql.functions import asc, desc

To specify sorting by multiple columns, we provide each column as a separate
argument to `.sort`.

In [116]:
mpg.sort(mpg.cyl.asc()).sort(mpg.hwy.desc()).show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|      toyota|   corolla|  1.8|1999|  4|manual(m5)|  f| 26| 35|  r|   compact|
|      toyota|   corolla|  1.8|2008|  4|  auto(l4)|  f| 26| 35|  r|   compact|
|       honda|     civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|      toyota|   corolla|  1.8|1999|  4|  auto(l4)| 

Here we will first reverse alphabetically by number of cylinders from lowest to highest, then by the vehicle's highway
mileage, from greatest to smallest.

## Grouping and Aggregating

To aggregate our data by group, we can use the `.groupBy` method. Like with
`.select`, we can pass either Column objects or strings that are column names
to `.groupBy`. All of the expressions below are equivalent.

In [118]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(F.col('cyl'))
mpg.groupBy('cyl')

Once the data is grouped, we need to specify an aggregation. We can use one of
the aggregate functions we imported earlier, alond with a column:

In [122]:
mpg.groupby('cyl').agg(F.round((F.avg(mpg.hwy)),2).alias('avg_hwy_mpg')).show()

+---+-----------+
|cyl|avg_hwy_mpg|
+---+-----------+
|  6|      22.82|
|  8|      17.63|
|  4|       28.8|
|  5|      28.75|
+---+-----------+



To group by multiple columns, pass each of the columns a a separate argument
to `.groupBy` (Note that this is different from pandas, where we would need to
pass a list).

In [128]:
# two level grop-by is passed as two positional arguments
# inside of the groupBy call
mpg.groupBy('cyl', 'trans').agg(F.avg(mpg.hwy)).show(5)

+---+--------+------------------+
|cyl|   trans|          avg(hwy)|
+---+--------+------------------+
|  6|auto(s6)|              26.0|
|  4|auto(s6)|             28.25|
|  8|auto(l4)|16.733333333333334|
|  4|auto(l5)|              31.0|
|  4|auto(av)|              30.5|
+---+--------+------------------+
only showing top 5 rows



In addition to `.groupBy`, we can use `.rollup`, which will do the same
aggregations, but will also include the overall total:

In [130]:
mpg.rollup('cyl', 'trans').agg(F.avg(mpg.hwy)).show()

+----+----------+------------------+
| cyl|     trans|          avg(hwy)|
+----+----------+------------------+
|   6|  auto(l5)|           21.4375|
|   6|  auto(av)|              26.0|
|   8|      null| 17.62857142857143|
|   6|manual(m6)|              22.6|
|   4|  auto(av)|              30.5|
|   6|      null| 22.82278481012658|
|   4|manual(m6)|29.571428571428573|
|   8|  auto(l4)|16.733333333333334|
|null|      null| 23.44017094017094|
|   6|  auto(s6)|              26.0|
|   4|      null| 28.80246913580247|
|   4|manual(m5)|29.272727272727273|
|   4|  auto(s6)|             28.25|
|   6|manual(m5)|22.666666666666668|
|   8|  auto(s6)|              20.4|
|   4|  auto(l5)|              31.0|
|   8|manual(m6)|              20.0|
|   6|  auto(l4)|22.689655172413794|
|   4|  auto(l4)|            27.625|
|   4|  auto(l3)|              27.0|
+----+----------+------------------+
only showing top 20 rows



Here the null value in `cyl` indicates the total count.

And in the example above, the null row represents the overall average highway
mileage.

## Crosstabs and Pivot Tables

In addition to groupby, spark provides a couple other ways to do aggregation.
One of which is `.crosstab`. This is very similary to pandas `.crosstab`
function, in that it calculates the number of occurances of each unique value
from the two passed columns:

In [131]:
mpg.crosstab('cyl', 'trans').show()

+---------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+
|cyl_trans|auto(av)|auto(l3)|auto(l4)|auto(l5)|auto(l6)|auto(s4)|auto(s5)|auto(s6)|manual(m5)|manual(m6)|
+---------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+
|        8|       0|       0|      30|      17|       4|       1|       1|       5|         5|         7|
|        5|       0|       0|       0|       0|       0|       0|       0|       2|         2|         0|
|        6|       3|       0|      29|      16|       2|       0|       1|       5|        18|         5|
|        4|       2|       2|      24|       6|       0|       2|       1|       4|        33|         7|
+---------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+



`.crosstab` simply does counts, if we want a different aggregation, we can use
`.pivot`. For example, to find the average highway mileage for each
combination of car class and number of cylinders, we could write the
following:

In [132]:
# so rather than passing two columns into our groupby
# in this instance, we will indeed chain the pivot 
# into the second method of our sequence here.
mpg.groupby('class').pivot('cyl').mean('hwy').show()

+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   compact|          29.46875|29.0|25.307692307692307|              null|
|   minivan|              24.0|null|              22.2|              null|
|       suv|             23.75|null|              18.5|16.789473684210527|
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   2seater|              null|null|              null|              24.8|
+----------+------------------+----+------------------+------------------+



Here the unique values from the column we group by will be the rows in the
resulting dataframe, and the unique values from the column we pivot on will
become the columns. The values in each cell will be equal to the aggregation
we specified over the group of values defined by the intersection of the rows
and the columns.

## Handling Missing Data

Let's take a look at how spark handles missing data. First we'll create a dataframe that has a few missing values:

In [135]:
df = session.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



Spark provides two main ways to deal with missing values:

- `.fill`: to replace missing values with a specified value
- `.drop`: to drop rows containing missing values

Both methods are accessed through the `.na` property. We'll look at some examples below:

In [136]:
df.na.fill(0).show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|0.0|
+---+---+



In [137]:
df.na.fill(0, subset='y').na.fill(1, subset='x').show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|1.0|0.0|
|4.0|3.0|
|5.0|1.0|
|1.0|0.0|
+---+---+



In [139]:
df.na.drop(subset='x').show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



Notice that above the na values in the `x` column were filled with 0, but the na values in y were left alone.

In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

In [140]:
df.na.fill(0, subset='x').na.drop(subset='y').show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



withColumn:

In [141]:
# withColumn is very similar to pd.DataFrame.assign()
# drop, as with na above, behaves similarly to pandas pd.DataFrame.drop()
mpg.withColumn('highway_again', F.col('hwy') + 1).drop('hwy').show()

+------------+------------------+-----+----+---+----------+---+---+---+-------+-------------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty| fl|  class|highway_again|
+------------+------------------+-----+----+---+----------+---+---+---+-------+-------------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18|  p|compact|           30|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21|  p|compact|           30|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20|  p|compact|           32|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21|  p|compact|           31|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16|  p|compact|           27|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18|  p|compact|           27|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18|  p|compact|           28|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)| 

## More Dataframe Manipulation Examples

Let's take a look at some more examples of working with spark dataframes. For
these examples, we'll be working with a dataset of observations of the
weather in seattle.

In [152]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = session.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



Let's print out the number of rows and columns in our dataset:

In [146]:
weather

DataFrame[date: string, precipitation: double, temp_max: double, temp_min: double, wind: double, weather: string]

In [145]:
weather.columns

['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']

In [144]:
# shape metrics of the spark df:
weather.count(), len(weather.columns)

(1461, 6)

Let's first find the dates where the data starts and stops:

In [147]:
min_date, max_date = F.min(weather.date), F.max(weather.date)
min_date, max_date

(Column<'min(date)'>, Column<'max(date)'>)

In [149]:
weather.select(min_date, max_date).first()

Row(min(date)='2012-01-01', max(date)='2015-12-31')

Here we use `.select` to select the minimum date and the maximum date.
`.first` returns us the first row of our results, which consists of two value,
and so can be unpacked into the `min_date` and `max_date` variables.

Next we will combine the temp max and min columns into a single column,
`temp_avg`.

In [154]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [160]:
# withColumn is similar to our df.assign()
weather = weather.withColumn(
'temp_avg',
    (weather.temp_min + weather.temp_max) / 2).drop(
'temp_min', 'temp_max')
# we did not reassign weather to the .show() version,
# which is essentially assigning a nonetype the same type of way
# that assigning a print statement would do
# we are assigning the set of changing instructions to weather
# so now when we pass show, these constructions apply as thus:

In [162]:
weather.dtypes

[('date', 'string'),
 ('precipitation', 'double'),
 ('wind', 'double'),
 ('weather', 'string'),
 ('temp_avg', 'double')]

In [161]:
weather.show(10)

+----------+-------------+----+-------+------------------+
|      date|precipitation|wind|weather|          temp_avg|
+----------+-------------+----+-------+------------------+
|2012-01-01|          0.0| 4.7|drizzle|               8.9|
|2012-01-02|         10.9| 4.5|   rain| 6.699999999999999|
|2012-01-03|          0.8| 2.3|   rain|              9.45|
|2012-01-04|         20.3| 4.7|   rain| 8.899999999999999|
|2012-01-05|          1.3| 6.1|   rain|              5.85|
|2012-01-06|          2.5| 2.2|   rain|3.3000000000000003|
|2012-01-07|          0.0| 2.3|   rain|               5.0|
|2012-01-08|          0.0| 2.0|    sun|               6.4|
|2012-01-09|          4.3| 3.4|   rain|               7.2|
|2012-01-10|          1.0| 3.4|   rain|3.3499999999999996|
+----------+-------------+----+-------+------------------+
only showing top 10 rows



Now we will calculate the total amount of rainfall for each month. We'll do
this by first creating a month column, then grouping by the month, and
finally, aggregating by taking the sum of the precipitation. To do this we will need to use the `month` function.

In [ ]:
# from pyspark.sql.functions import month, year, quarter

In [167]:
weather.withColumn(
    'month', F.month(F.col('date')
                    )
).groupBy(
    'month', 'weather'
).agg(
    F.sum(weather.precipitation).alias('total_rain')
).sort('total_rain').show()

+-----+-------+------------------+
|month|weather|        total_rain|
+-----+-------+------------------+
|    5|drizzle|               0.0|
|    1|drizzle|               0.0|
|    3|drizzle|               0.0|
|    6|drizzle|               0.0|
|    7|drizzle|               0.0|
|    2|drizzle|               0.0|
|   11|drizzle|               0.0|
|    8|drizzle|               0.0|
|   10|drizzle|               0.0|
|    9|drizzle|               0.0|
|   12|drizzle|               0.0|
|    9|   rain|0.8999999999999999|
|    4|drizzle|               1.0|
|    7|    fog|2.5999999999999996|
|    2|    sun|               4.0|
|    4|   snow|               4.6|
|    2|   snow|               5.7|
|    5|    sun|               6.1|
|    3|    sun| 8.899999999999999|
|   11|    sun|              16.2|
+-----+-------+------------------+
only showing top 20 rows



The `.sort` at the end isn't necessary, but presents that data in a friendlier
way.

Let's now take a look at the average temperature for each type of weather in
December 2013:

In [ ]:
# month will rip out the month numeral from the date
# establish a filter df[df.month == 12]
# pass a second filter of df[df.year == 2013]
# aggregate based on weather tag, present average temperature


Here we first have a couple of `.filter` calls in order to restrict our data
to December of 2013. We then group by the weather column, and lastly,
aggregate by taking the average of our `temp_avg` column. The combination of
group by and agg will calculate the average temperature for each unique value
of the `weather` column.

Let's now find out how many days had freezing temperatures in each month of
2013.

One last example, let's calculate the average temperature for each quarter of
each year:

Here we create the `quarter` and `year` columns, then group by these two new columns, and take the average temperature as our aggregate. Lastly, we sort by the year and quarter for presentation purposes.

We could also use a pivot table like this:

Here instead of grouping by two columns, we grouped by the first column and pivoted on the other column.

## Joins

Like pandas and sql, spark has functionality that lets us combine two tabular
datasets, known as a **join**.

We'll start by creating some data that we can join together:

In [169]:
users = session.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = session.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()

--- users ---
+---+-----+-------+
| id| name|role_id|
+---+-----+-------+
|  1|  bob|    1.0|
|  2|  joe|    2.0|
|  3|sally|    3.0|
|  4| adam|    3.0|
|  5| jane|    NaN|
|  6| mike|    NaN|
+---+-----+-------+

--- roles ---
+---+---------+
| id|     name|
+---+---------+
|  1|    admin|
|  2|   author|
|  3| reviewer|
|  4|commenter|
+---+---------+



To join two dataframes together, we'll need to call the `.join` method on one
of them and supply the other as an argument. In addition, we'll need to supply
the condition on which we are joining. In our case, we are joining where the
`role_id` column on the users table is equal to the `id` column on the roles
table.

In [171]:
users.join(roles, how='left', on=users.role_id == roles.id).show()

+---+-----+-------+----+--------+
| id| name|role_id|  id|    name|
+---+-----+-------+----+--------+
|  1|  bob|    1.0|   1|   admin|
|  2|  joe|    2.0|   2|  author|
|  3|sally|    3.0|   3|reviewer|
|  4| adam|    3.0|   3|reviewer|
|  5| jane|    NaN|null|    null|
|  6| mike|    NaN|null|    null|
+---+-----+-------+----+--------+



By default, spark will perform an inner join, meaning that records from both
dataframes will have a match with the other. We can also specify either a left
or a right join, which will keep all of the records from either the left or
right side, even if those records don't have a match with the other dataframe.

Notice that examples above have a duplicate `id` column. There are several
ways we could go about dealing with this:

- alias each dataframe + explicitly select columns after joining (this could also be implemented with spark SQL)
- rename duplicated columns before merging
- drop duplicated columns after the merge (`.drop(right.id)`)

## Visualization (or Lack Therof)

Spark does not provide a way to do visualization with their dataframes. To
visualize data from spark, you should use the `.toPandas` method on a spark
dataframe to convert it to a pandas dataframe, then visualize as you normally
would.

!!!warning "Converting to A Pandas Dataframe"
    Converting a spark dataframe to a pandas dataframe will pull all the data into memory, so make sure you have enough available memory to do so.

In [173]:
mpg.toPandas()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
229,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
230,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
231,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
232,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


## Exercises

Using the [repo setup directions](https://ds.codeup.com/fundamentals/git/), setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

1. Load the `mpg` dataset as a spark dataframe.

    1. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

    1. Transform the `trans` column so that it only contains either `manual` or `auto`.

1. Load the `tips` dataset as a spark dataframe.

    1. What percentage of observations are smokers?
    1. Create a column that contains the tip percentage
    1. Calculate the average tip percentage for each combination of sex and smoker.

1. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).